# Stellar EScrow Smart Contract

Anade Davis - Data Science Manager

Will Munson - Leader/Data Analyst

Harleen Kaur Bagga - Data Science Researcher

In [1]:
pip install stellar-sdk # This package is used to communicate with a Stellar Horizon serve=

     |████████████████████████████████| 412 kB 3.9 MB/s 
     |████████████████████████████████| 294 kB 56.7 MB/s 
     |████████████████████████████████| 178 kB 71.1 MB/s 
     |████████████████████████████████| 961 kB 45.6 MB/s 
     |████████████████████████████████| 62 kB 464 kB/s 
     |████████████████████████████████| 138 kB 60.1 MB/s 
     |████████████████████████████████| 1.3 MB 41.5 MB/s 
     |████████████████████████████████| 62 kB 643 kB/s 
     |████████████████████████████████| 59 kB 5.1 MB/s 
     |████████████████████████████████| 142 kB 77.3 MB/s 
  Created wheel for stellar-base-sseclient: filename=stellar_base_sseclient-0.0.21-py3-none-any.whl size=5490 sha256=13bc87defb8ff051f0b9716c0cd2c104e2ccb3059363ae68c30cb7fba821e8c4
  Stored in directory: /root/.cache/pip/wheels/a4/0f/aa/c8e47531a461e966fbc867a4ff00636ef16af7c1f726220993
Successfully built stellar-base-sseclient
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7

In [ ]:
from stellar_sdk import Keypair,Server,TransactionBuilder,Network,Asset,Signer



seller=Keypair.random()
buyer =  Keypair.random() 
escrow =Keypair.random()




In [ ]:
server = Server(horizon_url="https://horizon-testnet.stellar.org")


In [ ]:
import requests
response=requests.get(f"https://friendbot.stellar.org?addr={buyer.public_key}") #activate account of buyer
response=requests.get(f"https://friendbot.stellar.org?addr={escrow.public_key}")#activate account of escrow
response=requests.get(f"https://friendbot.stellar.org?addr={seller.public_key}")#activate account of seller



In [ ]:
# Now, let's plan out how we think this should work.  

#is the comment visible to you.
#Yes it is.

#To start off, we need to be able to sign into the eScrow account and fund it

# I think we need a pair that connects the buyer with the seller
# We need a keypair and an account
  


   
server = Server(horizon_url="https://horizon-testnet.stellar.org")
base_fee = server.fetch_base_fee() 
buyerKeyPair=Keypair.from_secret(buyer.secret)


buyer_account = server.load_account(account_id=buyer.public_key)
transaction = TransactionBuilder(
source_account=buyer_account,
network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
base_fee=base_fee) \
.append_create_account_op(destination=escrow.public_key, starting_balance="2.5000000") \
.build()
transaction.sign(buyerKeyPair)
response = server.submit_transaction(transaction)
print("Transaction hash: {}".format(response["hash"]))
print("New Keypair: \n\taccount id: {account_id}\n\tsecret seed: {secret_seed}".format(
account_id=escrow.public_key, secret_seed=escrow.secret))
  
    

    

  


    

    # Constructor for the public key
    # In the article I read, they set the starting balance at 2.500000, 

  # 

BadRequestError: ignored

In [ ]:
a=AccountCreation()
a.transaction_source()


Sending Lumens to the eScrow account: Here buyer funds the escrow account.

In [ ]:
''' paymentToEscrow():
     destination = escrow.public_key
     asset = Asset.native()
     amount = 10.000000 '''
class payThroughEscrow:
  # Buyer keypair
  buyerKeyPair=Keypair.from_secret(buyer.secret)
  base_fee = server.fetch_base_fee()
  buyerAccount=server.load_account(account_id=buyer.public_key)     
  def  sign_submitTransaction(self):
    transaction=(
        TransactionBuilder(
            source_account=buyerAccount,
            base_fee=base_fee,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE)
            .add_text_memo('Funding the escrow account')
            .append_payment_op(destination=escrow.public_key,asset_code = "XLM",amount = '10.000000')

            .set_timeout(100)
            .build()
        )
    response=transaction.sign(buyerKeyPair)#buyer signs the transaction  and sends the generated lumnens to seller
    xdr = transaction.to_xdr()
    print(f"xdr: {xdr}")
    response = server.submit_transaction(transaction)
    print(response)
    
    
    

In [ ]:
p=payThroughEscrow()
p.sign_submitTransaction()

Multisignature

In [ ]:
class eScrowMultisig:
  escrowKeyPair=Keypair.from_secret(escrow.secret)
  base_fee = server.fetch_base_fee()
  escrowAccount=server.load_account(account_id=escrow.public_key)
  '''Thresholds:
    masterWeight = 0 # Used for key, since it'll only be used once
    lowThreshold = 1 # Used for signing in
    midThreshold = 2 # Payment
    highThreshold = 2
    # The key will have a weight of 0, since it'll only ever be used once.
    # The threshold to sign in should be 1
    # The threshold to perform transactions should be 2: one for the buyer, one for the seller. '''

  # Create two signer functions, which both have weights of 1, 
  ''' Sellersigner particulars:
      ed25519PublicKey= seller.public_key,
      weight=1'''
  seller_signer = Signer.ed25519_public_key(account_id=seller.public_key, weight=1)    

  '''' Buyersigner particulars:
     ed25519PublicKey=buyer.public_key,
     weight= 1'''
  buyer_signer = Signer.ed25519_public_key(account_id=buyer.public_key, weight=1)    
  def  sign_submitTransaction(self): 
    transaction=(
        TransactionBuilder(
            source_account=escrowAccount,
            base_fee=base_fee,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE)
           .append_set_options_op(master_weight = 0 ,
           low_threshold = 1 ,
           med_threshold = 2 ,
           high_threshold = 2)
           .append_set_options_op(signer=seller_signer)
           .append_set_options_op(signer=buyer_signer)



            .set_timeout(100)
            .build()
        )
    transaction.sign(escrowKeyPair)
    response = server.submit_transaction(transaction)
    print(response)
    
   

In [ ]:
e=eScrowMultisig()
e.sign_submitTransaction()


### **Release Funds**

1. Create the eScrow dispursement transaction :Buyer creates an Escrow disbursement transaction to seller’s account
2.  Later on ,buyer signs the eScrow disbursement transaction.
3.  Finally seller signs the disbursement transaction and submits it to receive the funds.

In [ ]:
 

class releaseFunds:

   from stellar_sdk.transaction_envelope import TransactionEnvelope 
   base_fee = server.fetch_base_fee()
   '''Payment_To_Seller particulars:
   destination = seller.public_key
   asset = Asset.native()
   amount = 10.0000'''
   escrowAccount= server.load_account(account_id=escrow.public_key)  
   seller_secret=Keypair.from_secret(seller.secret)
  
   def  transaction(self):
    transaction=(
        TransactionBuilder(
            source_account=escrowAccount,
            base_fee=base_fee,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE)
            .add_text_memo('Release Funds to Seller')
            .append_payment_op(destination = seller.public_key,
     asset_code = "XLM",
     amount = '10.0000')

            .set_timeout(100)
            .build())
    #this involves the transaction from the eScrow account to the seller's account
    xdr=transaction.to_xdr( )
    print(f"xdr: {xdr}")
    network_passphrase = Network.TESTNET_NETWORK_PASSPHRASE

    buyer_secret = Keypair.from_secret(buyer.secret)
    network_passphrase = Network.TESTNET_NETWORK_PASSPHRASE



  

    # the buyer signs the disbursement transaction
    transaction = TransactionEnvelope.from_xdr(xdr, network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE)
    transaction.sign(buyer_secret)
    print(f"xdr: {transaction.to_xdr()}")
    resp = server.submit_transaction(transaction)
    print(resp)

    #Seller's private key can alse be used to sign the transaction and the signed transaction can be submitted to the network.
    transaction = TransactionEnvelope.from_xdr(xdr, network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE)
    transaction.sign(seller_secret)
    response = server.submit_transaction(transaction)
    print(response)


    







    
    
        
   
    


   

In [ ]:
r =  releaseFunds()
r.transaction()


Seller signs  and submits the transaction

Seller's private key can alse be used to sign the transaction and the signed transaction can be submitted to the network.

In [ ]:
#And the rest is history... 

# Similar to the buyer's signature, but for the seller

'''class releaseFundsToBuyer:
  sellerKeyPair = Keypair.from_secret(seller.secret)


 
  
  
  transaction = TransactionEnvelope.from_xdr(xdr,network_passphrase)
 

  #transaction = Transaction(envelope, Network.TESTNET)
  


  transaction.sign(sellerKeyPair)

  #await server.submitTransaction(transaction);'''

Just in case we need it: 

In [ ]:
# In case we need it

#class Start:


    
  #  const whoAreYou = [
  #    {
  #      type: 'list',
  #      name: 'whois',
  #      message: '¿Eres Alice o Bob?',
  #      choices: ['Alice', 'Bob']
  #    }
  
 
#print("Who are you ?")
#whois=input("If you are a buyer press 1 if you are a seller press 2")
  #if(whois == '1'):
    #def aliceActions():
      
  #if (whois === '1') {
      #print()
      
  #  const aliceActions = [
  #    {
  #     type: 'list',
  #     name: 'action',
  #     message: '¿Que quieres hacer?',
  #      choices: [
  #        {
  #          name: 'Crear el escrow',
  #          value: 'createEscrow'
  #        },
  #        {
  #          name: 'Añadir el mecanismo multifirma',
  #          value: 'setEscrowMultisig'
  #        },
  #        {
  #          name: 'Fondear el escrow',
  #          value: 'fundEscrow'
  #        },
  #        {
  #          name: 'Liberar los fondos',
  #          value: 'releaseFunds'
  #        },
  #        {
  #          name: 'Fimar la liberación de fondos',
  #          value: 'signFundsRelease'
  #        }
  #       ]
  #    }
  #  ];

    #  const { action } = await inquirer.prompt(aliceActions);

    #  switch (action) {
    #    case 'createEscrow':
    #      createEscrow();
    #      break;
    #    case 'setEscrowMultisig':
    #      setEscrowMultisig();
    #      break;
    #    case 'fundEscrow':
    #      fundEscrow();
    #      break;
    #    case 'releaseFunds':
    #      releaseFunds();
    #      break;
    #    case 'signFundsRelease':
    #      signFundsRelease():
    #      break;
    #  }
   # } else {
     # const bobActions = [
       # {
         #type: 'list',
         # name: 'action',
         # message: '¿Que quieres hacer?',
         # choices: [
           # {
          #    name: 'Retirar fondos',
         #     value: 'withdraw'
        #    }
       #   ]
      #  }
     # ];

    #  const { action } = await inquirer.prompt(bobActions);

   #   switch (action) {
   #     case 'withdraw':
   #       withdraw();
   #       break;
   #   }
   # }
  #} catch (err) {
  #  console.log(err);
 # }

#start();